# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [31]:
import pandas as pd

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

import re
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /home/sumit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sumit/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sumit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
#engine.table_names()
df = pd.read_sql_table('disaster_response',engine)
X = df['message']
y = df[df.columns[4:]]

In [3]:
for col in y.columns:
    print(col+":"+ str(y[col].sum()))

related:19906
request:4474
offer:118
aid_related:10860
medical_help:2084
medical_products:1313
search_and_rescue:724
security:471
military:860
water:1672
food:2923
shelter:2314
clothing:405
money:604
missing_people:298
refugees:875
death:1194
other_aid:3446
infrastructure_related:1705
transport:1201
buildings:1333
electricity:532
tools:159
hospitals:283
shops:120
aid_centers:309
other_infrastructure:1151
weather_related:7297
floods:2155
storm:2443
fire:282
earthquake:2455
cold:530
other_weather:1376
direct_report:5075


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    #remove urls
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    urls = re.findall(url_regex, text)
    
    for url in urls:
        text = text.replace(url, 'urlplaceholder')
    
    #Normalize text
    text = re.sub(r'[^a-zA-Z]', ' ', text.lower())
    
    #Tokenize text
    words = word_tokenize(text)
    
    #Get Rid of Stop Words
    words = [w for w in words if w not in stopwords.words('english')]
    
    #lemmatize words 
    wnl = WordNetLemmatizer()
    words = [wnl.lemmatize(w) for w in words]
    
    #lemmatize words (verbs)
    words = [wnl.lemmatize(w, pos='v') for w in words]
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [45]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('classifier',OneVsRestClassifier(AdaBoostClassifier(random_state=7)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=43, test_size=0.20)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x7f11df05bc10>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 OneVsRestClassifier(estimator=AdaBoostClassifier(algorith

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [48]:
# Predict and check performance
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred, target_names=y.columns.values))    

                        precision    recall  f1-score   support

               related       0.80      0.97      0.88      3984
               request       0.73      0.51      0.60       895
                 offer       0.00      0.00      0.00        24
           aid_related       0.75      0.61      0.68      2163
          medical_help       0.60      0.25      0.35       398
      medical_products       0.66      0.33      0.44       259
     search_and_rescue       0.62      0.19      0.29       136
              security       0.29      0.05      0.09        98
              military       0.63      0.36      0.46       184
                 water       0.73      0.66      0.69       322
                  food       0.79      0.65      0.71       581
               shelter       0.79      0.52      0.63       456
              clothing       0.66      0.42      0.51        84
                 money       0.54      0.29      0.38       131
        missing_people       0.25      

/home/sumit/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sumit/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [49]:
pipeline.get_params()

{'memory': None,
 'steps': [('vectorizer',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x7f11df05bc10>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('classifier',
   OneVsRestClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                    base_estimator=None,
                                                    learning_rate=1.0,
                                                    n_estimators=50,
                                                    random_state=7),
  

In [50]:
# Perform Grid Search for tuned parameters
params = {
          'tfidf__norm':['l1','l2'],
          'classifier__estimator__n_estimators': [30,50]
         }

cv = GridSearchCV(pipeline, param_grid= params, verbose=True)
cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 29.9min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vectorizer',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                   

In [51]:
# Checkout best parameters
cv.best_params_

{'classifier__estimator__n_estimators': 50, 'tfidf__norm': 'l2'}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [56]:
# Create pipeline again with tuned parameters
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer(norm='l2')),
    ('classifier',OneVsRestClassifier(AdaBoostClassifier(random_state=7, n_estimators=50)))    
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)  

In [57]:
print(classification_report(y_test, y_pred, target_names=y.columns.values)) 

                        precision    recall  f1-score   support

               related       0.80      0.97      0.88      3984
               request       0.73      0.51      0.60       895
                 offer       0.00      0.00      0.00        24
           aid_related       0.75      0.61      0.68      2163
          medical_help       0.60      0.25      0.35       398
      medical_products       0.66      0.33      0.44       259
     search_and_rescue       0.62      0.19      0.29       136
              security       0.29      0.05      0.09        98
              military       0.63      0.36      0.46       184
                 water       0.73      0.66      0.69       322
                  food       0.79      0.65      0.71       581
               shelter       0.79      0.52      0.63       456
              clothing       0.66      0.42      0.51        84
                 money       0.54      0.29      0.38       131
        missing_people       0.25      

/home/sumit/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sumit/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [67]:
# Test model
pd.DataFrame(pipeline.predict(['I need water and medicine']), columns=y.columns)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


### 9. Export your model as a pickle file

In [60]:
import pickle
pickle_file = open('../models/classifier.pkl', 'wb')
pickle.dump(pipeline, pickle_file)
pickle_file.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.